In [1]:
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import pylab as pl
import pylab
from pylab import *
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 12})
rc('text', usetex=True)
linecount = 512**3

In [2]:
def ToClass(X, NBIN, XMIN,XMAX):
    labels = (np.array(X-XMIN)/(XMAX-XMIN)*NBIN).astype(int)
    labels[labels < 0] = 0
    labels[labels >= NBIN] = NBIN-1
    return labels

def ToDM(X,NBIN,XMIN,XMAX):
    TEMP1 = (X*(XMAX-XMIN)/NBIN) + XMIN
    return TEMP1

In [3]:
# ======================================== SAVE DELTAS_HALO =============================================================
start_time = time.time()
linecount = 512**3
# pre-allocate array and load data into array
delta_R = np.zeros(linecount, dtype= np.float16)
with open('DELTA_R_01.txt','rb') as infile:
    for i,line in enumerate(infile):
        delta_R[i] = float(line)
    
print("--- %3fs: loaded ---" % (time.time() - start_time))

--- 114.413760s: loaded ---


In [4]:
# ======================================== SAVE DELTAS_DM ===============================================================
start_time = time.time()
# pre-allocate array and load data into array
delta_J = np.zeros(linecount, dtype= np.float16)
with open('DELTA_J_01.txt','rb') as infile:
    for i,line in enumerate(infile):
        delta_J[i] = float(line)
    
print("--- %3fs: loaded ---" % (time.time() - start_time))

--- 115.786719s: loaded ---


In [5]:
delta_J += 1.0
delta_R += 1.0
#delta_J_log = np.log10(delta_J)
#delta_R_log = np.log10(delta_R)

In [ ]:
# Transformacion DM en clases
def ToClass(X, NBIN, XMIN,XMAX):
    
    temp = (np.array(X-XMIN)/(XMAX-XMIN)*NBIN).astype(dtype=np.uint16)
    temp[temp < 0] = 0
    temp[temp >= NBIN] = NBIN-1
    return temp

# Transformacion clases a DM
def ToDM(X,NBIN,XMIN,XMAX):
    
    TEMP1 = (X*(XMAX-XMIN)/NBIN) + XMIN
    return TEMP1

# Muestra uniforme de DM
def Uniform_Sample (FLOATS, DM_data, F, POS):
    
    index = []
    Sample = []
    
    for i, List in enumerate(zip(FLOATS, DM_data, POS)):
        
        r = (List[0] < F[i]) # ===================================== EXTRAER LAS POSICIONES DE ALGUNA MANERA 
        Sample.append((List[1])[r])
        temp = np.array(List[2])
        index.append(temp[r])
        
    return Sample, index  

In [ ]:
# Ubicar cada DM en un bin
num_bins = int(raw_input("Num bins: "))
labels = ToClass(delta_J, num_bins,delta_J.min(),delta_J.max())
#print delta_J[:10], labels[:10], len(delta_J), len(labels)

In [ ]:
minval = delta_J.min()
maxval = delta_J.max()
delta = (maxval - minval) / num_bins
stop = int((20.0 - minval)/ delta  + 1 ) # Last bin of the sample

In [ ]:
random_values = np.random.rand(len(delta_J)).astype(dtype = np.float16) # Array de floats de largo 512^3  

In [ ]:
random_values_bins = [] 
DM_bins = []
Positions = []
for i in range(stop):
    List = np.where(labels == i)[0]                 # Posiciones de los bins iguales al valor i
    Positions.append(List)                          # Posiciones de los valores de DM correspondientes al bin i
    random_values_bins.append(random_values[List])  # Floats correpondientes al bin i 
    DM_bins.append(delta_J[List])                   # Valores de DM correspondientes a las posiciones señaladas en List
del(List)    

In [ ]:
N = np.zeros(stop)      # Array cuyas entradas son el número de datos que corresponden a labels == i
Frac = np.zeros(stop)   # Array cuyas entradas son límites superiores para seleccionar un porcentaje de la muestra total correspondiente a labels==i

for i in range(stop):
    N[i] = len(np.where(labels == i)[0]) #/ ((maxval - minval)/num_bins)
    if i == stop-1:
        MAX = float(N[stop-1])
        for j in range(stop):
            Frac[j] = MAX/N[j]
        

In [ ]:
DM_Sample, index =  Uniform_Sample(random_values_bins, DM_bins, Frac, Positions)

In [ ]:
del(random_values,N,Frac,labels)
del random_values_bins[:]
del DM_bins[:]

In [6]:
# 3D Arrays of zeros
deltas_DM = np.zeros((512,512,512), dtype = np.float16)
deltas_H = np.zeros((512,512,512), dtype = np.float16)

In [7]:
# Save all data in cubes of side 512
start_time = time.time()
for m in xrange(0,512**3):
    deltas_H[m % 512][(m/512) % 512][(m/512)/512] = delta_R[m] # Deltas Halo
    deltas_DM[m % 512][(m/512) % 512][(m/512)/512] = delta_J[m] # Deltas Dark Matter
    
del(delta_R)
del(delta_J)
print("--- %3fs: loaded ---" % (time.time() - start_time)) 

--- 405.806110s: loaded ---


In [ ]:
count = 0
for i in range(stop):
    for k in range(len(index[i])):
        count += 1
         
grid_position = np.zeros((count,3), dtype = np.uint16)
            
count2 = 0    
for i in range(stop):
    for j in range(len(index[i])):
        
        t = index[i][j]
        x,y,z = t%512, (t/512)%512, (t/512)/512
        grid_position[count2] = np.array([x,y,z])
        count2 += 1
    
    

In [ ]:
indexx = []
for i in range(len(grid_position)):
    r1 = np.any(grid_position[i,:] == 0)
    r2 = np.any(grid_position[i,:] == 511)
    if r1 == True or r2 == True:
        indexx.append(i)
ind = np.array(indexx)
GRID = np.delete(grid_position, ind, 0)
del(grid_position)

In [ ]:
data = np.zeros((len(GRID),28), dtype = np.float16) # delta_DM, neighbors
# For move in the 3D sub cube 
for m in range(len(GRID)): 
    x,y,z = GRID[m][0],GRID[m][1],GRID[m][2]
    data[m] = np.concatenate(([deltas_DM[x][y][z]],(deltas_H[x-1:x+2,y-1:y+2,z-1:z+2]).reshape(27)))
np.random.shuffle(data)                

In [ ]:
np.savetxt('RANDOM_SAMPLE.txt', data, fmt = '%.5f')    # Save data array on a file

In [ ]:
# Neighbors data and DM data center
data = np.zeros((100**3,29)) # True/False, delta_DM, neighbors

In [8]:
# SELECCION DE LOS DATOS DE PRUEBA 
start_time = time.time()
x,y,z= 100,100,100
tam = 200 # Sub cube of side tam
sub_arrayH = deltas_H[x:x+tam,y:y+tam,z:z+tam]
sub_arrayDM = deltas_DM[x:x+tam,y:y+tam,z:z+tam] # Save all data in the sub cube
arr = np.zeros((tam,tam,tam), dtype = bool)
#arr2 = np.zeros((tam,tam,tam), dtype = bool)
#arr3 = np.zeros((tam,tam,tam), dtype = bool)
i = 0 

# For move in the 3D sub cube 
for m1 in xrange(tam): 
    for m2 in xrange(tam):
        for m3 in xrange(tam):
                
                p1,p2,p3 = m1 + x, m2 + y, m3 + z 
                arr[m1][m2][m3] = np.all(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2] <= sub_arrayH[m1][m2][m3])  
                #arr2[m1][m2][m3] = np.median([deltas_H[p1-1][p2-1][p3-1], deltas_H[p1][p2-1][p3-1],deltas_H[p1+1][p2-1][p3-1],deltas_H[p1-1][p2][p3-1],deltas_H[p1][p2][p3-1],deltas_H[p1+1][p2][p3-1],deltas_H[p1-1][p2+1][p3-1],deltas_H[p1][p2+1][p3-1],deltas_H[p1+1][p2+1][p3-1],deltas_H[p1-1][p2-1][p3],deltas_H[p1][p2-1][p3],deltas_H[p1+1][p2-1][p3],deltas_H[p1-1][p2][p3],deltas_H[p1+1][p2][p3],deltas_H[p1-1][p2+1][p3],deltas_H[p1][p2+1][p3],deltas_H[p1+1][p2+1][p3],deltas_H[p1-1][p2-1][p3+1],deltas_H[p1][p2-1][p3+1],deltas_H[p1+1][p2-1][p3+1],deltas_H[p1-1][p2][p3+1],deltas_H[p1][p2][p3+1],deltas_H[p1+1][p2][p3+1],deltas_H[p1-1][p2+1][p3+1],deltas_H[p1][p2+1][p3+1],deltas_H[p1+1][p2+1][p3+1]]) < sub_arrayH[m1][m2][m3]
                #arr3[m1][m2][m3] = np.median(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2]) < sub_arrayH[m1][m2][m3]
                #data[i] = np.concatenate(([(arr[m1][m2][m3]).astype(int)],[sub_arrayDM[m1][m2][m3]],(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2]).reshape(27)))
                #DM_data[i] = sub_arrayDM[m1][m2][m3]
                #i += 1
                
print("--- %3fs: loaded ---" % (time.time() - start_time))

--- 186.846726s: loaded ---


In [ ]:
start_time = time.time()
ini = 200
tam = 200 # Sub cube of side tam
sub_arrayH = deltas_H[ini:ini+tam,ini:ini+tam,ini:ini+tam]
sub_arrayDM = deltas_DM[ini:ini+tam,ini:ini+tam,ini:ini+tam] # Save all data in the sub cube
count = 0

for m1 in xrange(tam): 
    for m2 in xrange(tam):
        for m3 in xrange(tam):
                
            p1,p2,p3 = m1 + ini, m2 + ini, m3 + ini 
            r = np.all(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2] <= sub_arrayH[m1][m2][m3])  
            if r == False:
                continue
            count += 1
            
print count
print("--- %3fs: loaded ---" % (time.time() - start_time))

In [ ]:
data_True = np.zeros((count,29), dtype = np.float16)
start_time = time.time()
ini = 200
tam = 200 # Sub cube of side tam
sub_arrayH = deltas_H[ini:ini+tam,ini:ini+tam,ini:ini+tam]
sub_arrayDM = deltas_DM[ini:ini+tam,ini:ini+tam,ini:ini+tam]
i = 0

for m1 in xrange(tam): 
    for m2 in xrange(tam):
        for m3 in xrange(tam):
                
            p1,p2,p3 = m1 + ini, m2 + ini, m3 + ini 
            r = np.all(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2] <= sub_arrayH[m1][m2][m3])  
            if r == False:
                continue
            data_True[i] =  np.concatenate(([1],[sub_arrayDM[m1][m2][m3]],(deltas_H[p1-1:p1+2,p2-1:p2+2,p3-1:p3+2]).reshape(27)))
            i += 1
            
print("--- %3fs: loaded ---" % (time.time() - start_time))            

In [ ]:
index = []
for i in range(len(data)):
    r = data[i][0] == 1
    if r == True:
        index.append(i)
ind = np.array(index)
data = np.delete(data,ind,0)
data = data[:(300000-count)]

index = []
for i in range(len(data)):
    r = np.any(data[i,2:] < 0)
    if r == True:
        index.append(i)
ind = np.array(index)
data_False = np.delete(data,ind,0)

In [ ]:
index = []
for i in range(len(data_True)):
    r = np.any(data_True[i,2:] < 0)
    if r == True:
        index.append(i)
ind = np.array(index)
data_True = np.delete(data_True,ind,0)

In [ ]:
DATA = np.concatenate((data_False, data_True))

In [ ]:
start_time = time.time()
formatt = []
formatt.append('%i')
for i in range(28):
    formatt.append('%.5f')
np.savetxt('TEST_DELTASS.txt', DATA, fmt = formatt)    # Save data array on a file
print("--- %3fs: loaded ---" % (time.time() - start_time))

In [9]:
# ===================================================== PLOTS ============================================================
start_time = time.time()

#import random
#ini = 100
#tam = 100 # Sub cube of side tam
#sub_arrayH = deltas_H[ini:ini+tam,ini:ini+tam,ini:ini+tam]
#sub_arrayDM = deltas_DM[ini:ini+tam,ini:ini+tam,ini:ini+tam]

#List1 = random.sample(range(len(sub_arrayH[arr == True])),3000)
#List2 = random.sample(range(len(sub_arrayH[arr == False])),3000)

import matplotlib.pyplot as plt

delta_DM_True = np.array(sub_arrayDM[arr == True])#[List1]
delta_H_True = np.array(sub_arrayH[arr == True])#[List1]

delta_DM_False = np.array(sub_arrayDM[arr == False])#[List2]
delta_H_False = np.array(sub_arrayH[arr == False])#[List2]

In [10]:
maxval, minval = 20, sub_arrayDM.min()
num_bins,num_bins_errorbar = 150,20
delta,delta_errobar = (maxval-minval) / num_bins , (maxval - minval) / num_bins_errorbar
i,l  = 0,0

labels_True = ToClass(delta_DM_True, num_bins, minval, maxval)
labels_False = ToClass(delta_DM_False, num_bins, minval, maxval)
#labels_True_errobar = ToClass(delta_DM_False, num_bins_errorbar, minval, maxval)
#labels_False_errobar = ToClass(delta_DM_False, num_bins_errorbar, minval, maxval)
bin_list_True = []
bin_list_False = []

while i < num_bins:
    r = np.any(labels_True == i)
    if r == True:
        bin_list_True.append(i)
    i += 1    
    
while l < num_bins:
    r = np.any(labels_False == l)
    if r == True:
        bin_list_False.append(l)
    l += 1    

bin_list_errobar_T = [k+1 for k in np.arange(0,num_bins,5)]
bin_list_errobar_F = [k+3 for k in np.arange(0,num_bins,5)]
    
a_0 = minval + delta/2
bins_True = np.array([ a_0 + delta*n for n in bin_list_True])
bins_False = np.array([ a_0 + delta*n for n in bin_list_False])
median_True = np.array([np.median((delta_H_True[labels_True == y])) for y in bin_list_True])
median_False = np.array([np.median((delta_H_False[labels_False == u])) for u in bin_list_False])
running_err_down_T    = [(np.sort(delta_H_True[labels_True == k]))[(len(delta_H_True[labels_True == k]))/5] for k in bin_list_errobar_T]
running_err_up_T      = [(np.sort(delta_H_True[labels_True == l]))[(4*len(delta_H_True[labels_True == l]))/5] for l in bin_list_errobar_T] 
running_err_down_F   = [(np.sort(delta_H_False[labels_False == k]))[(len(delta_H_False[labels_False == k]))/5] for k in bin_list_errobar_F]
running_err_up_F      = [(np.sort(delta_H_False[labels_False == l]))[(4*len(delta_H_False[labels_False == l]))/5] for l in bin_list_errobar_F] 
#np.savetxt('Median_T_10.txt',np.array([bins_True,median_True]).T, fmt = ['%.5f','%.5f'])
#np.savetxt('Median_F_10.txt',np.array([bins_False,median_False]).T, fmt = ['%.5f','%.5f'])

In [11]:
import random
List1 = random.sample(range(len(delta_DM_False)),2000)
List2 = random.sample(range(len(delta_DM_True)),2000)

In [22]:
plt.figure()
plt.plot(delta_DM_False[List1], delta_H_False[List1],'bo',alpha =.15, label =r"$ \delta_i > \delta_c$", markersize = 3.0)
plt.plot(delta_DM_True[List2], delta_H_True[List2],'ro',alpha =.15, label =r"$ \{\delta_i\} < \delta_c$", markersize = 3.0)
plt.xlabel(r"$\delta_{DM} + 1 $", fontsize = 16)
plt.ylabel(r"$\delta_{H} + 1 $", fontsize = 16) # '#00FF00'
plt.axis([0,10,0,10])
#plt.text(4.5, 3, r"Medians were adjust with $8*10^6$ overdensities", style='italic')
plt.title(r"Overdensity relations", fontsize = 16)
plt.plot(bins_True,median_True,color='red',linestyle = '-',lw=2.5,alpha=.7,label = r"Median $\left(\{\delta_i\}<\delta_c\right)$") 
plt.plot(bins_False,median_False,linestyle = '-',lw=2.5,alpha=.7,color = 'blue',label = r"Median $\left(\delta_i > \delta_c\right)$") 
plt.errorbar(bins_True[bin_list_errobar_T],median_True[bin_list_errobar_T], yerr = [median_True[bin_list_errobar_T]-running_err_down_T,running_err_up_T-median_True[bin_list_errobar_T]],alpha = .8, fmt = None, elinewidth = 1.5, ecolor = 'red')
plt.errorbar(bins_False[bin_list_errobar_F],median_False[bin_list_errobar_F], yerr = [median_False[bin_list_errobar_F]-running_err_down_F,running_err_up_F-median_False[bin_list_errobar_F]],alpha = .8, fmt = None, elinewidth = 1.5, ecolor = 'blue')
#plt.errorbar(bins,median, yerr = [median-running_err_down,running_err_up-median],alpha = 1.0, fmt = None, elinewidth = 1.2, ecolor = '#20b2aa')
pylab.legend(loc='lower right',frameon=True, prop={'size':14})
plt.show()

In [ ]:
# ===================================================== PLOTS ============================================================
start_time = time.time()
import random
sub_arrayH = deltas_H[100:200,100:200,100:200]
sub_arrayDM = deltas_DM[100:200,100:200,100:200]
#print (sub_arrayH[a1])[0:len(sub_arrayH[a1])]
#print sub_arrayH[a1]

#List1 = random.sample(range(len(sub_arrayH[arr == True])),3000)
#List2 = random.sample(range(len(sub_arrayH[arr == False])),3000)

import matplotlib.pyplot as plt

X1 = np.array(sub_arrayDM[arr == True])#[List1]
Y1 = np.array(sub_arrayH[arr == True])#[List1]

X2 = np.array(sub_arrayDM[arr == False])#[List2]
Y2 = np.array(sub_arrayH[arr == False])#[List2]

#print len(X1), len(X2)


total_bins = 10
#"""
plt.figure()
plt.plot(X1,Y1,'bo',alpha =.2, label =r"$\delta_i < \delta_c$", markersize = 3.0) # True
#plt.plot(X2,Y2,'ro',alpha =.2, label = r"$\{\delta_i\} > \delta_c$", markersize = 3.0) # False
plt.xlabel(r"$\delta_{DM}$", fontsize = 18)
plt.ylabel(r"$\delta_H$", fontsize = 18)
plt.axis([-1,10,-1,50])

bins1 = np.linspace(X1.min(),10, total_bins, endpoint = False)
delta1 = bins1[1]-bins1[0]
idx1  = np.digitize(X1,bins1)
#print Y1[idx == 6]
#print [np.median(Y1[idx == k]) for k in range(1,10)]
median1 = np.array([np.median(Y1[idx1 == k]) for k in range(1,total_bins+1)]) 
running_err1_down    = [(np.sort(Y1[idx1 == k]))[(len(Y1[idx1 == k]))/5] for k in range(1,total_bins+1)]
running_err1_up      = [(np.sort(Y1[idx1 == k]))[(4*len(Y1[idx1 == k]))/5] for k in range(1,total_bins+1)]

#print len(running_err1)

#bins2 = np.linspace(X2.min(),30, total_bins, endpoint = False)
#delta2 = bins2[1]-bins2[0]
#idx2  = np.digitize(X2,bins2)
#print Y1[idx == 6]
#print [np.median(Y1[idx == k]) for k in range(1,10)]
#median2 = np.array([np.median(Y2[idx2 == k]) for k in range(1,total_bins+1)])
#running_err2    = [[(np.sort(Y2[idx2 == k]))[(len(Y2[idx2 == k]))/5],(np.sort(Y2[idx2 == k]))[4*(len(Y2[idx2 == k]))/5]] for k in range(1,total_bins+1)]

plt.plot(bins1+delta1/2,median1,color='cyan',linestyle = '--',lw=2,alpha=.9,label = r"Median ($\delta_i < \delta_c$)") # True
plt.errorbar(bins1+delta1/2,median1, yerr = [running_err1_down-median1,median1-running_err1_up], fmt = None, ecolor = 'black')
#plt.plot(bins2+delta2/2,median2,color='magenta',linestyle = '--' ,lw=2,alpha=.9,label = r"Median ($\{\delta_i\} > \delta_c$)") # False
pylab.legend(loc='upper left',frameon=False, prop={'size':14})
plt.show()

del(sub_arrayH)
del(sub_arrayDM)

print("--- %3fs: loaded ---" % (time.time() - start_time))
#"""


In [ ]:
print (bins1+delta1/2)[0]
print median1[0]
print running_err1_down[0]
print running_err1_up[0]

In [ ]:
np.savetxt('True_points.txt',  np.array([X1,Y1]).T,)
np.savetxt('False_ points.txt', np.array([X2,Y2]).T)
np.savetxt('True_ median.txt',  np.array([bins1,median1]).T)
np.savetxt('False_ median.txt', np.array([bins2,median2]).T)

In [ ]:
A = np.random.rand(5,5)*10
print A,'\n'
print A[1:3,1:3]
B = np.all(A[1:3][1:3] < 7)
print B, '\n'
#print np.all(B)

In [ ]:

# count data rows, to preallocate array
f = open('DELTA_R_01.txt', 'rb')

def count(f):
    while 1:
        block = f.read()
        if not block:
             break
        yield block.count('\n')

linecount = sum(count(f))
print '\n%.3fs: file has %s rows' % (elapsed(), linecount)
f.close()

In [ ]:
import random

List = random.sample(range(1000000),10000)
import matplotlib.pyplot as plt

plt.plot(delta_J[List],delta_R[List],'ro')
plt.xlabel(r"$\delta_J$", fontsize = 18)
plt.ylabel(r"$\delta_R$", fontsize = 18)
plt.axis([0,30,0,30])
plt.show(

In [ ]:
import random
# ========= TEST ======================
i,r1,r2 = 0,True,True
while r1 == True and r2 == True and i < 10000:
    
    a = random.randrange(0,512)
    b = random.randrange(0,512)
    c = random.randrange(0,512)
    
    if deltas_DM[a][b][c] == delta_J[a + 512*b + (512**2)*c] and deltas_H[a][b][c] == delta_R[a + 512*b + (512**2)*c]:
        i += 1
        if i == 10000:
            print "OK!"
            


In [ ]:
A = np.random.rand(27).reshape((3,3,3))
print A
B = np.zeros(27)
mask = np.ones(len(B), dtype=bool)
mask[13] = False
B = ((A[0:3,0:3,0:3]).reshape(27))[mask]
print B



In [ ]:
# ======================================= HISTOGRAM DM DATA ==============================================================
import matplotlib.mlab as mlab

fig = plt.figure()

num_bins = 500

# the histogram of the data
n, bins, patches = plt.hist(delta_J, num_bins,log = True, facecolor='blue', alpha=0.5)

plt.xlabel(r"$ 1 + \delta_{H} $", fontsize = 16)
plt.ylabel(r"N", fontsize =16)
plt.show()

# r"$\log \left( 1 + \delta_{DM} \right) $"
# Tweak spacing to prevent clipping of ylabel
#plt.subplots_adjust(left=0.15)

#plot_url = py.plot_mpl(fig, filename='docs/histogram-mpl-legend')